<a href="https://colab.research.google.com/github/venkat2ram/ML_From_Scratch/blob/master/NeuralNet_From_Scratch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np


In [0]:
def init_params(layer_dims):
  np.random.seed(3)
  L=len(layer_dims)
  params={}
  for l in range(1,L):
    params['w'+str(l)]=np.random.randn(layer_dims[l],layer_dims[l-1])/0.01
    params['b'+str(l)]=np.zeros(layer_dim[l],1)
  return params


In [0]:
def sigmoid(Z):
  A=1/(1+np.exp(np.dot(-1,Z)))
  cache=(Z)
  return A,cache

In [0]:
def forward_prop(X,params):
  A=X
  caches=[]
  L=len(params)//2
  for l in range(1,L):
    A_prev=A
    Z=np.dot(params['w'+str(l)],A_prev)+params['b'+str(l)]
    linear_cache=(A_prev,params['w'+str(l)],params['b'+str(l)])
    A,activation_cache=sigmoid(Z)
    cache=(linear_cache,activation_cache)
    caches.append(cache)
  return A,caches   


**Rough**

In [26]:
 np.random.seed(3)
np.random.randn(2,2)

array([[ 1.78862847,  0.43650985],
       [ 0.09649747, -1.8634927 ]])

In [31]:
a=[1,2]
b=[[3,4],[5,6]]
c=[[3],[5]]
ca=(a,b)
ca=(ca,c)
ca

(([1, 2], [[3, 4], [5, 6]]), [[3], [5]])